<a href="https://colab.research.google.com/github/JPA-BERT/jpa-bert.github.io/blob/master/notebooks/05PyTorchTEXT_text_sentiment_ngrams_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
    
このファイルは PyTorch のチュートリアルにあるファイル <https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html> を翻訳して，加筆修正したもの
です。

すぐれたチュートリアルの内容，コードを公開された Sean Robertson と PyTorch 開発陣に敬意を表します。

- Original: 
- Date: 2020-0811
- Translated and modified: Shin Asakawa <asakawa@ieee.org>

---

In [5]:
# 2020年8月11日現在，以下のコマンドを実行してランタイムを再起動しなければ，このノートブックは動作しません。
# 理由は torchtext.datasets のバージョンが 0.3.0 であれば text_classification が定義されていないからです
# 以下のコマンドを用いて upgrade すると 0.7.0 にバージョンが更新され，動作するようになります。
!pip install --upgrade torchtext

     |████████████████████████████████| 4.5MB 13.6MB/s 
     |████████████████████████████████| 1.1MB 19.4MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [1]:
# from https://github.com/dmlc/xgboost/issues/1715
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
%matplotlib inline

## TorchText によるテキスト分類
<!--
## Text Classification with TorchText
-->

<!--This tutorial shows how to use the text classification datasets in ``torchtext``, including-->

このチュートリアルでは、 ``torchtext`` のテキスト分類データセットの使い方を説明します。

- AG_NEWS
- SogouNews
- DBpedia
- YelpReviewPolarity
- YelpReviewFull
- YahooAnswers
- AmazonReviewPolarity
- AmazonReviewFull

<!--
This example shows how to train a supervised learning algorithm for classification using one of these ``TextClassification`` datasets.
-->
ここでは 上記 ``TextClassification`` データセットを用いて，教師付き学習アルゴリズムによるテキスト分類でどのように訓練するかを示しています。

## ngrams によるデータの読み込み
<!--
## Load data with ngrams
-->

<!--A bag of ngrams feature is applied to capture some partial information about the local word order. 
In practice, bi-gram or tri-gram are applied to provide more benefits as word groups than only one word. An example:-->

局所的な語順についての部分的な情報を捕捉するため，ngrams 袋の特徴が適用されます。
(訳注: Bag of Words 単語袋とは，単語を語順に関わらず詰め込む袋に喩えて Bag of Words と言う)
実際には 1 つの単語だけよりも多くの利点を単語群として提供するため，バイグラム bigram (2-gram) または トライグラム (3-gram) が適用されます。
例としては

<!--
```
"load data with ngrams"
Bi-grams results: "load data", "data with", "with ngrams"
Tri-grams results: "load data with", "data with ngrams"
```
-->

```
"load data with ngrams"
Bi-grams 結果: "load data", "data with", "with ngrams"
Tri-grams 結果: "load data with", "data with ngrams"
```
<!--
``TextClassification`` Dataset supports the ngrams method. 
By setting ngrams to 2, the example text in the dataset will be a list of single words plus bi-grams string.
-->

``TextClassification`` データセットは ngrams メソッドをサポートしています。
ngrams を 2 に設定すると，データセットの例文は，単語のリスト と 2-grams の文字列になります。


In [3]:
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')

train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)

BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ag_news_csv.tar.gz: 11.8MB [00:00, 22.6MB/s]
120000lines [00:07, 16086.20lines/s]
120000lines [00:16, 7314.09lines/s]
7600lines [00:01, 7524.96lines/s]


In [4]:
torchtext.__version__

'0.7.0'

## モデルの定義
<!--
## Define the model
-->

<!--
The model is composed of the
[EmbeddingBag](https://pytorch.org/docs/stable/nn.html?highlight=embeddingbag#torch.nn.EmbeddingBag) layer and the linear layer (see the figure below). 
-->

モデルは，[埋め込み袋 EmbeddingBag](https://pytorch.org/docs/stable/nn.html?highlight=embeddingbag#torch.nn.EmbeddingBag) 層と線形層から構成されています。
下図を参照してください。

<!--
``nn.EmbeddingBag`` computes the mean value of a “bag” of embeddings. The text entries here have different lengths. 
``nn.EmbeddingBag`` requires no padding here since the text lengths are saved in offsets.
-->

``nn.EmbeddingBag`` は埋め込み「袋」の平均値を計算します。ここでのテキスト項目の長さはそれぞれ異なります。
``nn.EmbeddingBag`` は埋め込み (padding) を必要としません。テキストの長さはオフセット（ズレ）で保存されからです。

<!--
Additionally, since ``nn.EmbeddingBag`` accumulates the average across the embeddings on the fly, ``nn.EmbeddingBag`` can enhance the performance and memory efficiency to process a sequence of tensors.
-->

さらに ``nn.EmbeddingBag`` は，その場でその都度埋め込み全体の平均値を蓄積します。
従って ``nn.EmbeddingsBag`` はテンソル系列を処理する性能とメモリ効率が向上します。

<!--
![](https://github.com/pytorch/tutorials/blob/gh-pages/_downloads/_static/img/text_sentiment_ngrams_model.png?raw=1)
-->

<img src="https://pytorch.org/tutorials/_images/text_sentiment_ngrams_model.png">




In [5]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## 事例の初期化
<!--
## Initiate an instance
-->

<!--The AG_NEWS dataset has four labels and therefore the number of classes is four.-->
AG_NEWS データセット は 4 つのラベルを持っており，クラス数は 4つです。

```
1 : World 世界
2 : Sports スポーツ
3 : Business ビジネス
4 : Sci/Tec 科学工学
```

<!--
The vocab size is equal to the length of vocab (including single word and ngrams). 
The number of classes is equal to the number of labels, which is four in AG_NEWS case.
-->
語彙サイズは，語彙の長さ （語彙長）に等しくなります(単語と ngramsを含みます）
クラス数はラベル数に等しく，AG_NEWS の場合は 4 です。


In [6]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

## バッチ生成に用いる関数
<!--
## Functions used to generate batch
-->

<!--
Since the text entries have different lengths, a custom function generate_batch() is used to generate data batches and offsets. 
The function is passed to ``collate_fn`` in ``torch.utils.data.DataLoader``.
The input to ``collate_fn`` is a list of tensors with the size of batch_size, and the ``collate_fn`` function packs them into a mini-batch. 
Pay attention here and make sure that ``collate_fn`` is declared as a top level def. 
This ensures that the function is available in each worker.
-->

テキストエントリの長さはそれぞれ異なるので，データのバッチとオフセットを生成するためにカスタム関数 `generate_batch()` を用います。
この関数は ``torch.utils.data.DataLoader``  の ``collate_fn`` に渡されます。
``collate_fn`` への入力は `batch_size` サイズのテンソルリストです。 ``collate_fn``関数はそれらをミニバッチにまとめます。
ここで注意してほしいのは，``collate_fn`` がトップレベルの関数 (def) として宣言されていることです。
これにより，各ワーカーでこの関数が利用できるようになります。

<!--
The text entries in the original data batch input are packed into a list and concatenated as a single tensor as the input of ``nn.EmbeddingBag``.
The offsets is a tensor of delimiters to represent the beginning index of the individual sequence in the text tensor. 
Label is a tensor saving the labels of individual text entries.
-->

元のデータ一括入力のテキストエントリはリストに詰められ ``nn.EmbeddingBag`` の入力として 1 つのテンソルとして連結されます。
オフセット(ずれ) は，テキストテンソル内の個々の系列の開始インデックスを表す分割子（デリミタ） のテンソルです。
ラベル (Label) は，個々のテキスト項目のラベルを保存したテンソルです。


In [7]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

## モデルの訓練と結果の評価のための関数の定義
<!--
## Define functions to train the model and evaluate results.
-->

<!--
[torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader) is recommended for PyTorch users, and it makes data loading in parallel easily (a tutorial is [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html). 
We use ``DataLoader`` here to load AG_NEWS datasets and send it to the model for training/validation.
-->

[torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader) は PyTorch ユーザにおすすめのツールです。
これにより，データの読み込みを簡単に並列して行うことができます。チュートリアルは[こちら](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)。
ここでは AG_NEWS データセットをロードしてモデルに送り，訓練や検証を行うために ``DataLoader`` を使用します。


In [8]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

## データセットの分割とモデルの実行
<!--
## Split the dataset and run the model
-->

<!--
Since the original AG_NEWS has no valid dataset, we split the training dataset into train/valid sets with a split ratio of 0.95 (train) and
0.05 (valid). 
Here we use [torch.utils.data.dataset.random_split](https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split) function in PyTorch core library.
-->

元の AG_NEWS には検証データセットがないため，学習データセットを 分割率 0.95 (学習データ) で学習/有効 データセットに分割します 0.05 は検証データセット。
ここでは PyTorch コアライブラリ [torch.utils.data.dataset.random_split](https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split) 関数を使用しています。

<!--
[CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss) criterion combines nn.LogSoftmax() and nn.NLLLoss() in a single class.
It is useful when training a classification problem with C classes. 
[SGD](https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html) implements stochastic gradient descent method as optimizer. 
The initial learning rate is set to 4.0.
[StepLR](https://pytorch.org/docs/master/_modules/torch/optim/lr_scheduler.html#StepLR) is used here to adjust the learning rate through epochs. 
-->

[交差エントロピー損失 CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss) 基準は `nn.LogSoftmax()` (訳注: 対数ソフトマックス) と `nn.NLLLoss()` (訳注: 負の対数尤度) を一つのクラスにまとめたものです。
C クラスを用いた分類問題を学習する際に便利です。
[確率的勾配降下法 SGD](https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html) はオプティマイザとして確率的勾配降下法を実装しています。
初期学習率は 4.0 に設定されています。
ここでは [StepLR](https://pytorch.org/docs/master/_modules/torch/optim/lr_scheduler.html#StepLR) を用いてエポック単位で学習率を調整しています。


In [9]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 8 seconds
	Loss: 0.0261(train)	|	Acc: 84.7%(train)
	Loss: 0.0000(valid)	|	Acc: 88.7%(valid)
Epoch: 2  | time in 0 minutes, 8 seconds
	Loss: 0.0120(train)	|	Acc: 93.6%(train)
	Loss: 0.0000(valid)	|	Acc: 91.5%(valid)
Epoch: 3  | time in 0 minutes, 9 seconds
	Loss: 0.0068(train)	|	Acc: 96.5%(train)
	Loss: 0.0001(valid)	|	Acc: 91.6%(valid)
Epoch: 4  | time in 0 minutes, 8 seconds
	Loss: 0.0039(train)	|	Acc: 98.1%(train)
	Loss: 0.0000(valid)	|	Acc: 91.2%(valid)
Epoch: 5  | time in 0 minutes, 8 seconds
	Loss: 0.0023(train)	|	Acc: 99.0%(train)
	Loss: 0.0001(valid)	|	Acc: 91.7%(valid)


<!--
Running the model on GPU with the following information:
-->
GPU を用いてモデルを実行すると以下のよう情報を得ます

```
Epoch: 1 | time in 0 minutes, 11 seconds
       Loss: 0.0263(train)     |       Acc: 84.5%(train)
       Loss: 0.0001(valid)     |       Acc: 89.0%(valid)

Epoch: 2 | time in 0 minutes, 10 seconds
       Loss: 0.0119(train)     |       Acc: 93.6%(train)
       Loss: 0.0000(valid)     |       Acc: 89.6%(valid)

Epoch: 3 | time in 0 minutes, 9 seconds
       Loss: 0.0069(train)     |       Acc: 96.4%(train)
       Loss: 0.0000(valid)     |       Acc: 90.5%(valid)

Epoch: 4 | time in 0 minutes, 11 seconds
       Loss: 0.0038(train)     |       Acc: 98.2%(train)
       Loss: 0.0000(valid)     |       Acc: 90.4%(valid)


Epoch: 5 | time in 0 minutes, 11 seconds
       Loss: 0.0022(train)     |       Acc: 99.0%(train)
       Loss: 0.0000(valid)     |       Acc: 91.0%(valid)
```


## テストデータを用いたモデルの評価
<!--
## Evaluate the model with test dataset
-->

In [10]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0002(test)	|	Acc: 90.6%(test)


テストデータセットの結果をチェック:

<!--
Checking the results of test dataset…
-->

```
       Loss: 0.0237(test)      |       Acc: 90.5%(test)
```


## ランダムニューズでのテスト
<!--
## Test on a random news
-->

<!--
Use the best model so far and test a golf news. The label information is available [here](https://pytorch.org/text/datasets.html?highlight=ag_news#torchtext.datasets.AG_NEWS).
-->

今までの最良モデルを使ってゴルフニュースを試してみてください。
ラベル情報は[こちら](https://pytorch.org/text/datasets.html?highlight=ag_news#torchtext.datasets.AG_NEWS)。



In [ ]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news


これはスポーツニュースです
<!--This is a Sports news-->



<!--
You can find the code examples displayed in this note [here](https://github.com/pytorch/text/tree/master/examples/text_classification) 
-->

このノートのコードは [こちら](https://github.com/pytorch/text/tree/master/examples/text_classification) です。